In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
#dataset_name = "prostate_cancer"
dataset_name = "animal_faces"
#model_from = "task1"
model_from = "pretrained"

In [ ]:
dataset_url = "/content/drive/MyDrive/MLProject/Codes/Task2/Features/features_labels_"+model_from+"_model_"+dataset_name+".csv"
dataset = pd.read_csv(dataset_url)

In [ ]:
print(dataset.shape)
print(dataset.head)

(6000, 513)
<bound method NDFrame.head of              0         1         2         3         4         5         6  \
0     1.332117  0.962382  1.172957  1.880547  1.254007  0.145058  0.072821   
1     1.478649  1.711469  0.125946  4.729920  1.682038  0.304509  0.232022   
2     1.465653  0.690341  0.383253  3.801867  1.036663  0.728186  0.000000   
3     1.911302  1.174756  0.684350  3.891245  2.204816  0.384372  0.550843   
4     0.948928  0.597579  0.311711  1.867843  2.095466  0.307914  0.810284   
...        ...       ...       ...       ...       ...       ...       ...   
5995  2.152292  1.830096  0.107098  4.205874  3.133372  2.415588  0.258176   
5996  4.125968  1.075508  0.095616  6.483240  0.612922  0.857934  0.013475   
5997  2.697565  1.215197  0.661574  1.982565  2.004938  0.000000  0.018319   
5998  2.119469  1.727767  1.295557  1.941219  1.192652  0.780768  0.139652   
5999  0.943357  1.731140  0.731666  1.766039  1.134306  2.167492  0.116700   

             7       

In [ ]:
# Separate features and labels
X = dataset.iloc[:, :-1]  # Features (all columns except the last one)
y = dataset.iloc[:, -1]   # Labels (last column)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
knn_classifier = KNeighborsClassifier()

In [ ]:
param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
}

In [ ]:
# Perform grid search using GridSearchCV
grid_search = GridSearchCV(knn_classifier, param_grid, cv=10, scoring='accuracy', return_train_score=True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             return_train_score=True, scoring='accuracy')

In [ ]:
train_accuracy = grid_search.cv_results_['mean_train_score']
test_accuracy = grid_search.cv_results_['mean_test_score']

In [ ]:
grid_search_cv_results = pd.DataFrame(columns=['hyperparameters', 'train_accuracy', 'test_accuracy'])

In [ ]:
for params, train_acc, test_acc in zip(grid_search.cv_results_['params'], train_accuracy, test_accuracy):
    print(f"Model with Hyperparameters: {params}")
    print(f"Training Accuracy: {train_acc:.4f}")
    print(f"Testing Accuracy: {test_acc:.4f}\n")
    new_row = pd.Series({'hyperparameters': params, 'train_accuracy': train_acc, 'test_accuracy': test_acc})
    grid_search_cv_results = grid_search_cv_results.add(new_row, ignore_index=True)

Model with Hyperparameters: {'n_neighbors': 3, 'p': 1, 'weights': 'uniform'}
Training Accuracy: 0.9573
Testing Accuracy: 0.9281

Model with Hyperparameters: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
Training Accuracy: 1.0000
Testing Accuracy: 0.9281

Model with Hyperparameters: {'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
Training Accuracy: 0.9616
Testing Accuracy: 0.9310

Model with Hyperparameters: {'n_neighbors': 3, 'p': 2, 'weights': 'distance'}
Training Accuracy: 1.0000
Testing Accuracy: 0.9310

Model with Hyperparameters: {'n_neighbors': 5, 'p': 1, 'weights': 'uniform'}
Training Accuracy: 0.9489
Testing Accuracy: 0.9274

Model with Hyperparameters: {'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
Training Accuracy: 1.0000
Testing Accuracy: 0.9281

Model with Hyperparameters: {'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
Training Accuracy: 0.9519
Testing Accuracy: 0.9326

Model with Hyperparameters: {'n_neighbors': 5, 'p': 2, 'weights': 'distance'}
Training Accurac

In [ ]:
grid_search_cv_results.to_csv("/content/drive/MyDrive/MLProject/Codes/Task2/Results/grid_search_cv_results_KNN_"+model_from+"_model_"+dataset_name+".csv",index=False)

In [ ]:
with open('/content/drive/MyDrive/MLProject/Codes/Task2/Results/grid_search_results_KNN_'+model_from+'_model_'+dataset_name+'.txt', 'w') as file:
    file.write("Hyperparameters\tTrain_Accuracy\tTest_Accuracy\n")
    for params, train_acc, test_acc in zip(grid_search.cv_results_['params'], train_accuracy, test_accuracy):
        file.write(f"{params}\t{train_acc:.4f}\t{test_acc:.4f}\n")

In [ ]:
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)


Best Hyperparameters: {'n_neighbors': 5, 'p': 2, 'weights': 'distance'}


In [ ]:
best_model = grid_search.best_estimator_

with open('/content/drive/MyDrive/MLProject/Codes/Task2/Models/KNN_best_model_config_'+model_from+'_model_'+dataset_name+'.txt', 'w') as file:
    file.write("Best Hyperparameters:\n")
    file.write(best_params)

In [ ]:
# Assess the performance on training data
y_train_pred = best_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_conf_matrix = confusion_matrix(y_train, y_train_pred)
train_classification_rep = classification_report(y_train, y_train_pred)

In [ ]:
print("Training Data Results:")
print("Accuracy:", train_accuracy)
print("Confusion Matrix:")
print(train_conf_matrix)
print("Classification Report:")
print(train_classification_rep)

Training Data Results:
Accuracy: 1.0
Confusion Matrix:
[[1399    0    0]
 [   0 1415    0]
 [   0    0 1386]]
Classification Report:
              precision    recall  f1-score   support

       gland       1.00      1.00      1.00      1399
    nongland       1.00      1.00      1.00      1415
       tumor       1.00      1.00      1.00      1386

    accuracy                           1.00      4200
   macro avg       1.00      1.00      1.00      4200
weighted avg       1.00      1.00      1.00      4200



In [ ]:
# Save training results to a file
with open('/content/drive/MyDrive/MLProject/Codes/Task2/Results/training_results_Best_KNN_'+model_from+'_model_'+dataset_name+'.txt', 'w') as file:
    file.write("Training Data Results:\n")
    file.write(f"Accuracy: {train_accuracy}\n")
    file.write("Confusion Matrix:\n")
    file.write(f"{train_conf_matrix}\n")
    file.write("Classification Report:\n")
    file.write(f"{train_classification_rep}\n")

In [ ]:
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_conf_matrix = confusion_matrix(y_test, y_test_pred)
test_classification_rep = classification_report(y_test, y_test_pred)

In [ ]:
print("\nTesting Data Results:")
print("Accuracy:", test_accuracy)
print("Confusion Matrix:")
print(test_conf_matrix)
print("Classification Report:")
print(test_classification_rep)


Testing Data Results:
Accuracy: 0.9255555555555556
Confusion Matrix:
[[561  38   2]
 [ 32 553   0]
 [ 61   1 552]]
Classification Report:
              precision    recall  f1-score   support

       gland       0.86      0.93      0.89       601
    nongland       0.93      0.95      0.94       585
       tumor       1.00      0.90      0.95       614

    accuracy                           0.93      1800
   macro avg       0.93      0.93      0.93      1800
weighted avg       0.93      0.93      0.93      1800



In [ ]:
with open('/content/drive/MyDrive/MLProject/Data/Task2/Results/testing_results_Best_KNN_'+model_from+'_model_'+dataset_name+'.txt', 'w') as file:
    file.write("Testing Data Results:\n")
    file.write(f"Accuracy: {test_accuracy}\n")
    file.write("Confusion Matrix:\n")
    file.write(f"{test_conf_matrix}\n")
    file.write("Classification Report:\n")
    file.write(f"{test_classification_rep}\n")